# RNN

### LSTM and GRU

generating text

In [29]:
from __future__ import print_function
from keras.layers import Dense , Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
import numpy as np

In [7]:
data_file  = open("11-0.txt",'rb')
lines =[]
for line in data_file:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    if len(line) == 0:
           continue
    lines.append(line)
data_file.close()
text = " ".join(lines)

In [10]:
chars = set([c for c in text])
nb_chars = len(chars)
chars2index = dict((c,i) for i,c in enumerate(chars))
index2chars = dict((i,c) for i,c in enumerate(chars))

In [17]:
SEQLEN = 10
STEP   = 1
input_chars = []
label_chars = []
for i in range(0,len(text)-SEQLEN , STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])
    

In [19]:
input_chars[0],label_chars[0]

(u'project gu', u't')

In [22]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
        for j, ch in enumerate(input_char):
            X[i, j, chars2index[ch]] = 1
        y[i, chars2index[label_chars[i]]] = 1


In [25]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [27]:
model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,input_shape=(SEQLEN, nb_chars),
           unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy",optimizer="rmsprop")

In [32]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # testing model
    # randomly choose a row from input_chars, then use it to 
    # generate text from model for next 100 chars
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, chars2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2chars[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
    print()


Iteration #: 0
Epoch 1/1
158773/158773 [==============================] - 8s 51us/step - loss: 1.9362
Generating from seed: while in s
while in she had hare the mous the mous the mous the mous the mous the mous the mous the mous the mous the mou
Iteration #: 1
Epoch 1/1
158773/158773 [==============================] - 8s 52us/step - loss: 1.8550
Generating from seed:  there wer
 there were the pook of the pook of the pook of the pook of the pook of the pook of the pook of the pook of th
Iteration #: 2
Epoch 1/1
158773/158773 [==============================] - 8s 52us/step - loss: 1.7904
Generating from seed: ng the lea
ng the leas it a coust it was the could the mouse the could the mouse the could the mouse the could the mouse 
Iteration #: 3
Epoch 1/1
158773/158773 [==============================] - 8s 51us/step - loss: 1.7386
Generating from seed: he project
he project gutenberg-tm encersation as it and the doon as the catting it a dore the liget it on the project gu
Iteration #: 4
E